In [3]:
import numpy as np
import numpy.linalg as la

In [2]:
data = np.array([[1, 2, 1, 2, 10, 10.3, 10.5, 10.7],
                 [1, 1, 2, 2,  2,  2,  2, 2]])
labels = np.array([[-1, -1, 1, 1, 1, 1, 1, 1]])
blue_th = np.array([[0, 1]]).T
blue_th0 = -1.5
red_th = np.array([[1, 0]]).T
red_th0 = -2.5

In [4]:
def magnitude(th):
    return la.norm(th)

def margin(x, y, th, th0):
    return y*(th.T@x+th0)/magnitude(th)

In [14]:
def score_sum(data, labels, th, th0):
    _, n = data.shape
    margin_sum = 0
    for i in range(n):
        margin_sum += margin(data[:, i:i+1], labels[:, i:i+1], th, th0)
    return margin_sum[0, 0]

def score_min(data, labels, th, th0):
    _, n = data.shape
    margin_min = margin(data[:, 0:1], labels[:, 0:1], th, th0)[0, 0]
    for i in range(1, n):
        margin_min = min(margin_min, margin(data[:, i:i+1], labels[:, i:i+1], th, th0)[0, 0])
    return margin_min

def score_max(data, labels, th, th0):
    _, n = data.shape
    margin_max = margin(data[:, 0:1], labels[:, 0:1], th, th0)[0, 0]
    for i in range(1, n):
        margin_max = max(margin_max, margin(data[:, i:i+1], labels[:, i:i+1], th, th0)[0, 0])
    return margin_max


In [15]:
print(score_sum(data, labels, red_th, red_th0))
print(score_min(data, labels, red_th, red_th0))
print(score_max(data, labels, red_th, red_th0))

31.5
-1.5
8.2


In [16]:
print(score_sum(data, labels, blue_th, blue_th0))
print(score_min(data, labels, blue_th, blue_th0))
print(score_max(data, labels, blue_th, blue_th0))

4.0
0.5
0.5


In [21]:
data = np.array([[1.1, 1, 4],[3.1, 1, 2]])
labels = np.array([[1, -1, -1]])
th = np.array([[1, 1]]).T
th0 = -4

In [25]:
def hinge_loss(x, y, th, th0, ref):
    m = margin(x, y, th, th0)
    if m < ref:
        return 1 - m/ref
    return 0

In [26]:
for i in range(3):
    print(hinge_loss(data[:, i:i+1], labels[:, i:i+1], th, th0, 1/2**0.5))

[[0.8]]
0
[[3.]]
